In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import accuracy_score
import random as rand
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import time

In [4]:
data = pd.read_csv("/content/drive/MyDrive/Sem 1/Data Mining/HW/DATASET/HW3 Dataset/data.csv").to_numpy()
labels = pd.read_csv("/content/drive/MyDrive/Sem 1/Data Mining/HW/DATASET/HW3 Dataset/label.csv",header = None).to_numpy().flatten()

In [28]:
def random_centroids(data, K):
    centroids = []

    for i in range(K):
        centroid = data[rand.randint(0, 149)]
        centroids.append(centroid)
    return centroids


def assign_cluster(data, centroids):
    assignments = []

    for data_point in data:
        dist_point_clust = []

        for centroid in centroids:
            d_clust = np.linalg.norm(np.array(data_point) - np.array(centroid))
            dist_point_clust.append(d_clust)

        assignment = np.argmin(dist_point_clust)
        assignments.append(assignment)

    return assignments

def new_centroids(data, centroids, assignments, K):
    new_centroids = []
    for i in range(K):
        pt_cluster = []
        for x in range(len(data)):
                if (assignments[x] == i):
                    pt_cluster.append(data[x])
        mean_c = np.mean(pt_cluster, axis=0)
        new_centroids.append(mean_c)

    return new_centroids

def sse(data, assignments, centroids):
    errors = []

    for i in range(len(data)):

        centroid = centroids[assignments[i]]
        error = np.linalg.norm(np.array(data[i]) - np.array(centroid))
        errors.append(error**2)

    sse = sum(errors)

    return sse

K-means Execution and Evaluation:



*   Iterates over each distance function.
*   Calls the k_means function for the current distance function and records the resulting cluster labels, centroids, and SSE.
*   Evaluates clustering performance by comparing the obtained cluster labels with the true labels (labels) and calculates accuracy.
Stores SSE and Accuracy in the results dictionary.


In [42]:
def euclidean_distance(x1, x2):
    return np.linalg.norm(x1 - x2)

def cosine_similarity(x1, x2):
    dot_product = np.dot(x1, x2)
    norm_x1 = np.linalg.norm(x1)
    norm_x2 = np.linalg.norm(x2)
    similarity = dot_product / (norm_x1 * norm_x2)
    return 1 - similarity

def jaccard_similarity(x1, x2):
    intersection = np.sum(np.minimum(x1, x2))
    union = np.sum(np.maximum(x1, x2))
    similarity = intersection / union
    return 1 - similarity


def k_means(data, k, distance_function, max_iters=100):
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    labels = np.zeros(n_samples)

    for _ in range(max_iters):
        for i in range(n_samples):
            distances = [distance_function(data[i], centroid) for centroid in centroids]
            labels[i] = np.argmin(distances)


        for j in range(k):
            mask = labels == j
            centroids[j] = np.mean(data[mask], axis=0)

    sse = sum([distance_function(data[i], centroids[int(labels[i])])**2 for i in range(n_samples)])

    return labels, centroids, sse

data = pd.read_csv("/content/drive/MyDrive/Sem 1/Data Mining/HW/DATASET/HW3 Dataset/data.csv",header = None).to_numpy()
labels = pd.read_csv("/content/drive/MyDrive/Sem 1/Data Mining/HW/DATASET/HW3 Dataset/label.csv",header = None).to_numpy()

# Set the number of clusters (k)
k = len(np.unique(labels))  # Number of categories in labels

distance_functions = [euclidean_distance, cosine_similarity, jaccard_similarity]
distance_function_names = ['Euclidean', 'Cosine', 'Jaccard']

results = {}


for distance_function, distance_function_name in zip(distance_functions, distance_function_names):

    cluster_labels, cluster_centroids, sse = k_means(data, k, distance_function)
    accuracy = np.sum(cluster_labels == labels.squeeze()) / len(labels)
    results[distance_function_name] = {'SSE': sse, 'Accuracy': accuracy}

for method, result in results.items():
    print(f"{method} K-means - SSE: {result['SSE']:.2f}, Accuracy: {result['Accuracy'] * 100:.2f}%")


Euclidean K-means - SSE: 25517481494.00, Accuracy: 13.39%
Cosine K-means - SSE: 697.81, Accuracy: 5.59%
Jaccard K-means - SSE: 3653.15, Accuracy: 7.95%


Q1 - Compare SEE


1. Euclidean K-means - SSE: 25517481494.00, Accuracy: 13.39%
2. Cosine K-means - SSE: 697.81, Accuracy: 5.59%
3. Jaccard K-means - SSE: 3653.15, Accuracy: 7.95%.

We see that Cosine Distance function has the lowest sum of squared errors

Majority Vte Execution and Evaluation
* Iterates over each distance function.
* Calls the k_means_majority_vote function for the current distance function and records the accuracy.
* Stores accuracy in the accuracies dictionary.

In [43]:
data = pd.read_csv("/content/drive/MyDrive/Sem 1/Data Mining/HW/DATASET/HW3 Dataset/data.csv", header=None).to_numpy()
labels = pd.read_csv("/content/drive/MyDrive/Sem 1/Data Mining/HW/DATASET/HW3 Dataset/label.csv", header=None).to_numpy().flatten()

def k_means_majority_vote(data, k, distance_function, max_iters=100):
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    assigned_labels = np.zeros(n_samples)

    for _ in range(max_iters):
        for i in range(n_samples):
            distances = [distance_function(data[i], centroid) for centroid in centroids]
            assigned_labels[i] = np.argmin(distances)

        for j in range(k):
            mask = assigned_labels == j
            centroids[j] = np.mean(data[mask], axis=0)

    # Label each cluster using majority vote
    cluster_labels = np.zeros(k)
    for j in range(k):
        cluster_mask = assigned_labels == j
        majority_label = np.argmax(np.bincount(labels[cluster_mask].astype(int)))
        cluster_labels[j] = majority_label

    # Assign cluster labels to data points
    predicted_labels = np.array([cluster_labels[int(label)] for label in assigned_labels])

    accuracy = np.sum(predicted_labels == labels.squeeze()) / len(labels)

    return accuracy


distance_functions = [euclidean_distance, cosine_similarity, jaccard_similarity]
distance_function_names = ['Euclidean', 'Cosine', 'Jaccard']


accuracies = {}

for distance_function, distance_function_name in zip(distance_functions, distance_function_names):

    accuracy = k_means_majority_vote(data, k, distance_function)
    accuracies[distance_function_name] = accuracy

for method, accuracy in accuracies.items():
    print(f"{method} K-means with Majority Vote - Accuracy: {accuracy * 100:.2f}%")


Euclidean K-means with Majority Vote - Accuracy: 59.72%
Cosine K-means with Majority Vote - Accuracy: 57.37%
Jaccard K-means with Majority Vote - Accuracy: 60.12%


Q2 We observe the following results


1.   Euclidean K-means with Majority Vote - Accuracy: 59.72%
2.   Cosine K-means with Majority Vote - Accuracy: 57.37%
3.   Jaccard K-means with Majority Vote - Accuracy: 60.12%

The choice of distance metric can significantly impact the performance of clustering algorithms. The effectiveness of a distance metric depends on the characteristics of the data and the underlying structure of the clusters.

In [44]:
import time
# K-means algorithm with stop criteria
def k_means_stop_criteria(data, k, distance_function, max_iters=500, tolerance=1e-4):
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    assigned_labels = np.zeros(n_samples)

    for iter_count in range(max_iters):
        for i in range(n_samples):
            distances = [distance_function(data[i], centroid) for centroid in centroids]
            assigned_labels[i] = np.argmin(distances)

        new_centroids = np.array([np.mean(data[assigned_labels == j], axis=0) for j in range(k)])

        # Check stop criteria
        if np.all(np.abs(new_centroids - centroids) < tolerance):
            break
        centroids = new_centroids

    sse = sum([distance_function(data[i], centroids[int(assigned_labels[i])])**2 for i in range(n_samples)])

    return assigned_labels, centroids, iter_count + 1, sse


distance_functions = [euclidean_distance, cosine_similarity, jaccard_similarity]
distance_function_names = ['Euclidean', 'Cosine', 'Jaccard']
results = {}

# Apply K-means algorithm with stop criteria for each distance function
for distance_function, distance_function_name in zip(distance_functions, distance_function_names):
    start_time = time.time()

    # Run K-means with stop criteria
    cluster_labels, cluster_centroids, num_iterations, sse = k_means_stop_criteria(data, k, distance_function)

    end_time = time.time()

    # Store information in the dictionary
    results[distance_function_name] = {
        'Cluster Labels': cluster_labels,
        'Centroids': cluster_centroids,
        'Num Iterations': num_iterations,
        'SSE': sse,
        'Time to Converge': end_time - start_time
    }

# Display results
for method, result in results.items():
    print(f"{method} K-means - Iterations: {result['Num Iterations']}, SSE: {result['SSE']:.2f}, Time to Converge: {result['Time to Converge']:.4f} seconds")


Euclidean K-means - Iterations: 48, SSE: 25392039606.17, Time to Converge: 57.5660 seconds
Cosine K-means - Iterations: 59, SSE: 682.22, Time to Converge: 124.8682 seconds
Jaccard K-means - Iterations: 69, SSE: 3660.35, Time to Converge: 182.2073 seconds


Q3 The result we are :
1. Euclidean K-means - Iterations: 48, SSE: 25392039606.17, Time to Converge: 57.5660 seconds
2. Cosine K-means - Iterations: 59, SSE: 682.22, Time to Converge: 124.8682 seconds
3. Jaccard K-means - Iterations: 69, SSE: 3660.35, Time to Converge: 182.2073 seconds

We see that Euclidean Kmeans take the least amount iterations and the time to coverge, Euclidean distance is sensitive to the scale of features. The high SSE may indicate that the algorithm struggles to converge when dealing with data that has varying feature scales.

In [45]:
# K-means algorithm with stop criteria
def k_means_stop_criteria(data, k, distance_function, max_iters=100, tolerance=1e-4):
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    assigned_labels = np.zeros(n_samples)

    for iter_count in range(max_iters):
        for i in range(n_samples):
            distances = [distance_function(data[i], centroid) for centroid in centroids]
            assigned_labels[i] = np.argmin(distances)

        new_centroids = np.array([np.mean(data[assigned_labels == j], axis=0) for j in range(k)])

        # Check stop criteria
        if np.all(np.abs(new_centroids - centroids) < tolerance):
            break

        # Update centroids for the next iteration
        centroids = new_centroids


    sse = sum([distance_function(data[i], centroids[int(assigned_labels[i])])**2 for i in range(n_samples)])

    return assigned_labels, centroids, iter_count + 1, sse


distance_functions = [euclidean_distance, cosine_similarity, jaccard_similarity]
distance_function_names = ['Euclidean', 'Cosine', 'Jaccard']


sse_results = {}

for distance_function, distance_function_name in zip(distance_functions, distance_function_names):
    # Run K-means with stop criteria
    cluster_labels, cluster_centroids, num_iterations, sse = k_means_stop_criteria(data, k, distance_function)

    sse_results[distance_function_name] = sse

# Display SSEs
for method, sse in sse_results.items():
    print(f"{method} K-means - SSE: {sse:.2f}")

Euclidean K-means - SSE: 25373275133.23
Cosine K-means - SSE: 684.57
Jaccard K-means - SSE: 3708.24


Q4 The results that we see are
1. Euclidean K-means - SSE: 25373275133.23
2. Cosine K-means - SSE: 684.57
3. Jaccard K-means - SSE: 3708.24

SSE values reflect how well the K-means algorithm is able to minimize the sum of squared distances between data points and their assigned cluster centroids. The choice of distance function significantly influences the clustering performance, and the characteristics of the data, such as feature scales and inherent structure, play a crucial role in determining the SSE values for each distance metric.

Q5
* Euclidean distance, being sensitive to feature scale, leads to suboptimal clustering results.
* Cosine and Jaccard distance metrics, while having lower SSE, still exhibit challenges in capturing the underlying cluster structure.
* The use of majority voting improves accuracy in all cases.
* Euclidean K-means converges faster than Cosine and Jaccard K-means with stop criteria.